In [1]:
#A lot of code from: https://ai.google.dev/gemma/docs/core/huggingface_text_finetune_qlora
%run verifyHuggingFacePkgs.py
#check if bfloat or float
if torch.cuda.get_device_capability()[0] >= 8:
    dtype = torch.bfloat16
    print("nice")
else:
    dtype = torch.float16

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


--- Bitsandbytes GPU Verification Script ---

Step 1: Checking for CUDA-enabled GPU...
✅ Success: CUDA is available. Found GPU: Orin

Step 2: Loading a model with 8-bit quantization (`load_in_8bit=True`)...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


✅ Success: Model loaded in 8-bit without errors.
   This indicates that bitsandbytes is correctly installed and communicating with the GPU.

Step 3: Verifying model properties...
   - Model is on device: cuda:0
   ✅ Model is correctly placed on the CUDA device.
   - Model memory footprint: 165.54 MB

Step 4: Performing a simple inference test (forward pass)...
✅ Success: Forward pass completed without errors.

--- Verification Complete ---
🎉 All checks passed! Your `bitsandbytes` installation appears to be working correctly with your GPU.
nice


In [2]:
import os
import sys
# Get the absolute path of the current script's directory
current_dir = os.path.dirname(os.path.abspath("gemini2.5.ipynb"))

# Get the absolute path of the parent directory (project_folder)
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to the Python path
sys.path.append(parent_dir)

# Now you can import from GetXY.py
from GetXY import x_string, y

# ... rest of your code
print("Successfully imported variables!")

-2 - 1 + -3
2543
-6.0

Expressions not in x:
-4 + -5 + -1
True
1457
-10.0
15
-4.0
[-5.   1.   1.   0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5]
Successfully imported variables!


In [3]:
y_string = []
for entry in y: 
    y_string.append(str(entry))
print(y_string[0])

-6.0


In [4]:
import datasets
print("success")

success


In [5]:
def create_conversation(x_list, y_list):
    conversations = []
    for x, y in zip(x_list, y_list):
        conversations.append({
            "messages": [
                {"role": "user", "content": x},
                {"role": "assistant", "content": y}
            ]
        })
    return conversations

# System message for the assistant
#system_message = ""

# User prompt that combines the user query and the schema
#user_prompt = ""


dataset = create_conversation(x_string, y_string)
print(dataset[0])
from datasets import Dataset
dataset = Dataset.from_list(dataset)
print(dataset["messages"][0])

{'messages': [{'role': 'user', 'content': '-2 - 1 + -3'}, {'role': 'assistant', 'content': '-6.0'}]}
[{'content': '-2 - 1 + -3', 'role': 'user'}, {'content': '-6.0', 'role': 'assistant'}]


In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "google/gemma-3-270m-it"

# Define model init arguments
model_kwargs = dict(
    attn_implementation="eager", # Use "flash_attention_2" when running on Ampere or newer GPU
    dtype=dtype, # What torch dtype to use, defaults to auto
    device_map="auto", # Let torch decide how to load the model
)

# BitsAndBytesConfig: Enables 4-bit quantization to reduce model size/memory usage
model_kwargs["quantization_config"] = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=model_kwargs['dtype'],
    bnb_4bit_quant_storage=model_kwargs['dtype'],
)

model = AutoModelForCausalLM.from_pretrained(model_id, **model_kwargs)
tokenizer = AutoTokenizer.from_pretrained(model_id)



In [7]:
if tokenizer.pad_token_id is None:
    print("new")
    tokenizer.pad_token_id = tokenizer.eos_token_id

#tokenizer.apply_chat_template(dataset, tokenize = False)
def format_chat_template(example):
    conversation = example['messages']
    formatted_text = tokenizer.apply_chat_template(
        conversation,          # <-- Pass the list of messages for ONE conversation
        tokenize=False,
        add_generation_prompt=False # False for training examples
    )
    return {'text': formatted_text}
formatted_dataset = dataset.map(format_chat_template)

Map:   0%|          | 0/2543 [00:00<?, ? examples/s]

In [8]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=16,
    bias="none",
    target_modules="all-linear",
    task_type=None,
    modules_to_save=["lm_head", "embed_tokens"] # make sure to save the lm_head and embed_tokens as you train the special tokens
)


In [9]:
from trl import SFTConfig

args = SFTConfig(
    output_dir="output",         # directory to save and repository id
    max_length=512,                         # max sequence length for model and packing of the dataset
    packing=True,                           # Groups multiple samples in the dataset into a single sequence
    num_train_epochs=3,                     # number of training epochs
    per_device_train_batch_size=1,          # batch size per device during training
    gradient_accumulation_steps=4,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch",              # use fused adamw optimizer
    logging_steps=10,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    fp16=True if dtype == torch.float16 else False,   # use float16 precision
    bf16=True if dtype == torch.bfloat16 else False,   # use bfloat16 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="cosine",           # use constant learning rate scheduler
    push_to_hub=True,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
    dataset_kwargs={
        "add_special_tokens": False, # We template with special tokens
        "append_concat_token": True, # Add EOS token as separator token between examples
    }
)


In [10]:
from trl import SFTTrainer

# Create Trainer object
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    peft_config=peft_config,
    processing_class=tokenizer
)

Padding-free training is enabled, but the attention implementation is not set to 'flash_attention_2'. Padding-free training flattens batches into a single sequence, and 'flash_attention_2' is the only known attention mechanism that reliably supports this. Using other implementations may lead to unexpected behavior. To ensure compatibility, set `attn_implementation='flash_attention_2'` in the model configuration, or verify that your attention mechanism can handle flattened sequences.
You are using packing, but the attention implementation is not set to 'flash_attention_2' or 'kernels-community/vllm-flash-attn3'. Packing flattens batches into a single sequence, and Flash Attention is the only known attention mechanisms that reliably support this. Using other implementations may lead to cross-contamination between batches. To avoid this, either disable packing by setting `packing=False`, or set `attn_implementation='flash_attention_2'` or `attn_implementation='kernels-community/vllm-flash

Tokenizing train dataset:   0%|          | 0/2543 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/2543 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': 2, 'pad_token_id': 0}.


Step,Training Loss


In [ ]:
trainer.save_model()